<center><h1>Preprocessing Pipeline</h1></center>

In [12]:
import pandas as pd

data = {"Name":["Anna", "Bob", "Charlie", "Diana", "Eric"], #notice the names are in alphabetical order
        "Age":[20, 34, 23, None, 33],
        "Gender":["f", "m", "m", "f", "m"],
        "Job": ["Programmer", "Writer", "Cook", "Programmer", "Teacher"]}

df= pd.DataFrame(data)

In [13]:
df

,Name,Age,Gender,Job
0,Anna,20.0,f,Programmer
1,Bob,34.0,m,Writer
2,Charlie,23.0,m,Cook
3,Diana,NaN,f,Programmer
4,Eric,33.0,m,Teacher


Preprocessing Pipeline:

* Drop Name Feature
* Imputer Ages
* Turn Gender into Binary / Numeric
* One Hot Encode Jobs

In [14]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Drop Name
df = df.drop(["Name"], axis=1)

# Impute Ages
imputer = SimpleImputer(strategy = "mean")
df["Age"] = imputer.fit_transform(df[["Age"]])

# Numeric Gender
gender_dct = {"m":0, "f":1}
df["Gender"] = [gender_dct[g] for g in df["Gender"]]

# OneHotEncode Jobs

encoder = OneHotEncoder()
matrix = encoder.fit_transform(df[['Job']]).toarray()
column_names = sorted([i for i in df['Job'].unique()])

for i in range (len(matrix.T)):
    df[column_names[i]] = matrix.T[i]

df = df.drop(["Job"], axis=1)

In [15]:
df

,Age,Gender,Cook,Programmer,Teacher,Writer
0,20.0,1,0.0,1.0,0.0,0.0
1,34.0,0,0.0,0.0,0.0,1.0
2,23.0,0,1.0,0.0,0.0,0.0
3,27.5,1,0.0,1.0,0.0,0.0
4,33.0,0,0.0,0.0,1.0,0.0


<h1>Creating the pipeline</h1>

In [42]:
from sklearn.base import BaseEstimator, TransformerMixin

class NameDropper (BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform (self, X):
        return X.drop(['Name'], axis = 1)

class AgeImputer (BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform (self, X):
        imputer = SimpleImputer (strategy = "mean")
        X['Age'] = imputer.fit_transform (X[['Age']])
        return X

class FeatureEncoder (BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform (self, X):
        gender_dct = {"m":0, "f":1}
        X['Gender'] = [gender_dct[g] for g in X['Gender']]
    
        encoder = OneHotEncoder()
        matrix = encoder.fit_transform(X[['Job']]).toarray()
        
        column_names = sorted([i for i in X['Job'].unique()])
        
        for i in range (len(matrix.T)):
            X[column_names[i]] = matrix.T[i]
    
        return X.drop(['Job'], axis=1)

In [43]:
data2 = {"Name":["Fionna", "Gerald", "Hans", "Isabella", "Jacob"],
        "Age":[20, 34, None, None, 33],
        "Gender":["f", "m", "m", "f", "m"],
        "Job": ["Writer", "Programmer", "Programmer", "Programmer", "Teacher"]}
df2 = pd.DataFrame(data2)
df2

,Name,Age,Gender,Job
0,Fionna,20.0,f,Writer
1,Gerald,34.0,m,Programmer
2,Hans,NaN,m,Programmer
3,Isabella,NaN,f,Programmer
4,Jacob,33.0,m,Teacher


In [44]:
dropper = NameDropper()
imp = AgeImputer ()
enc = FeatureEncoder()

enc.fit_transform(imp.fit_transform(dropper.fit_transform(df2)))

,Age,Gender,Programmer,Teacher,Writer
0,20.0,1,0.0,0.0,1.0
1,34.0,0,1.0,0.0,0.0
2,29.0,0,1.0,0.0,0.0
3,29.0,1,1.0,0.0,0.0
4,33.0,0,0.0,1.0,0.0


In [45]:
from sklearn.pipeline import Pipeline

pipe = Pipeline ([
    ("dropper", NameDropper()),
    ("imputer", AgeImputer()),
    ("encoder", FeatureEncoder())
])

pipe.fit_transform(df2)

,Age,Gender,Programmer,Teacher,Writer
0,20.0,1,0.0,0.0,1.0
1,34.0,0,1.0,0.0,0.0
2,29.0,0,1.0,0.0,0.0
3,29.0,1,1.0,0.0,0.0
4,33.0,0,0.0,1.0,0.0
